# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [113]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

In [114]:
# Dictionary to map the text rating class names to integer values
RATING_MAP = {
    'One': 1, 
    'Two': 2, 
    'Three': 3, 
    'Four': 4, 
    'Five': 5
}

def get_numeric_rating(tag):
    """
    Extracts the rating class from the HTML tag and converts it to an integer.
    Example: 'star-rating Three' -> 3
    """
    # Ratings are stored in a class attribute, e.g., class="star-rating Three"
    rating_class = tag.find('p')['class'][-1] if tag.find('p') else 'Zero'
    return RATING_MAP.get(rating_class, 0)


In [123]:
def scrape_books(min_rating: int, max_price: float) -> pd.DataFrame:
    """
    Scrapes book data from Books to Scrape, filters the results, and returns a DataFrame.

    Parameters:
    - min_rating (int): The minimum acceptable rating (1 to 5).
    - max_price (float): The maximum acceptable price.

    Returns:
    - pandas.DataFrame: A filtered DataFrame of book data.
    """
    BASE_URL ="https://books.toscrape.com/catalogue/"
    current_page_url = BASE_URL + 'page-1.html'
    all_books_data = []
    page_count = 0

    while current_page_url:
        page_count += 1
        print(f"Scraping Page {page_count}: {current_page_url}")

        try:
            response = requests.get(current_page_url, timeout=10)
            response.raise_for_status() # Raise an exception for bad status codes (4xx or 5xx)
        except requests.exceptions.RequestException as e:
            print(f"Error fetching page {current_page_url}: {e}")
            break

        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Select all book article containers on the page
        book_articles = soup.find_all('article', class_='product_pod')

        if not book_articles:
            print("No more books found on this page. Stopping.")
            break
        for article in book_articles:
            # 1. Title
            title_tag = article.find('h3').find('a')
            title = title_tag['title'] if title_tag else 'N/A'
            detail_link = title_tag['href']
            
            # Construct the absolute detail URL
            detail_url = requests.compat.urljoin(BASE_URL, detail_link)

            # 2. Price (Extract, clean, and convert to float)
            price_tag = article.find("p", attrs={"class":"price_color"})
            price_match = re.search(r'[\d\.]+', price_tag.text) if price_tag else None
            price_float = float(price_match.group(0)) if price_match else 0.0
            
            # 3. Rating (Convert class name to integer)
            rating = get_numeric_rating(article)

            # 4. Availability
            availability = article.find("p", attrs={"class":"instock availability"}).get_text().strip() if article.find("p", attrs={"class":"instock availability"}) else 'N/A'
            
            # --- STAGE 2: Scrape Detail Page for Genre ---
            genre = "N/A"
            try:
                detail_response = requests.get(detail_url, timeout=5)
                detail_response.raise_for_status()
                detail_soup = BeautifulSoup(detail_response.text, 'html.parser')
                
                # The category/genre is the second link in the breadcrumb list
                breadcrumb_items = detail_soup.select('ul.breadcrumb li a')
                if len(breadcrumb_items) > 1:
                    # Index 0 is 'Home', Index 1 is the Category/Genre
                    genre = breadcrumb_items[1].text
                
                #Getting description
                descrip=detail_soup.select('article.product_page p')
                description=descrip

                # Getting UPC
                get_upc=detail_soup.select("tr td")
                upc=get_upc[0]
                # Be polite: wait a small amount of time between detail page requests
                time.sleep(0.1) 
                
            except requests.exceptions.RequestException as e:
                print(f"    Error fetching detail page {detail_url}. Genre set to 'Error'.")
                # print(f"    Error: {e}") # Uncomment for debugging
                genre = "Error"

            book_info = {
                'Title': title,
                'Price': price_float,
                'Rating': rating,
                'Availability': availability,
                'Genre': genre,
                "Description": description,
                "UPC": upc
            }
            all_books_data.append(book_info)
            
        # Find the next page link for pagination
        next_button = soup.find('li', class_='next')
        if next_button:
            # The 'href' is relative, so we append it to the base URL
            next_url = next_button.find('a')['href']
            # For the first page, the URL changes from index.html to page-2.html, 
            # so we ensure we prepend the base path correctly.
            if 'index.html' in current_page_url:
                current_page_url = BASE_URL + next_url
                time.sleep(0.5)
            else:
                # If we are on page-X.html, we need to join the relative path correctly
                current_page_url = requests.compat.urljoin(current_page_url, next_url)
                
            
        else:
            current_page_url = None

    print("\n--- Scraping Complete. Starting Filtering. ---")

    # Create the DataFrame from the collected data
    df = pd.DataFrame(all_books_data)
    
    # --- Filtering Logic ---
    # Filter by minimum rating & by maximum price
    df_filtered = df[(df['Rating'] >= min_rating) & (df['Price'] <= max_price)]
    return df_filtered
    print(f"Total books scraped: {len(df)}")





In [124]:
# Define your filtering parameters
MIN_RATING_THRESHOLD = 4  # Only show books rated 4 stars or higher
MAX_PRICE_THRESHOLD = 20.00 # Only show books priced at £20.00 or less

# Run the scraper
filtered_df = scrape_books(min_rating=MIN_RATING_THRESHOLD, max_price=MAX_PRICE_THRESHOLD)

Scraping Page 1: https://books.toscrape.com/catalogue/page-1.html
Scraping Page 2: https://books.toscrape.com/catalogue/page-2.html
Scraping Page 3: https://books.toscrape.com/catalogue/page-3.html
Scraping Page 4: https://books.toscrape.com/catalogue/page-4.html
Scraping Page 5: https://books.toscrape.com/catalogue/page-5.html
Scraping Page 6: https://books.toscrape.com/catalogue/page-6.html
Scraping Page 7: https://books.toscrape.com/catalogue/page-7.html
Scraping Page 8: https://books.toscrape.com/catalogue/page-8.html
Scraping Page 9: https://books.toscrape.com/catalogue/page-9.html
Scraping Page 10: https://books.toscrape.com/catalogue/page-10.html
Scraping Page 11: https://books.toscrape.com/catalogue/page-11.html
Scraping Page 12: https://books.toscrape.com/catalogue/page-12.html
Scraping Page 13: https://books.toscrape.com/catalogue/page-13.html
Scraping Page 14: https://books.toscrape.com/catalogue/page-14.html
Scraping Page 15: https://books.toscrape.com/catalogue/page-15.htm

In [100]:
filtered_df


NameError: name 'filtered_df' is not defined

In [63]:

"""
product[0].find("img")["alt"]
def get_brand(item):
    brand_name=item.find("img")["alt"]
    return brand_name
"""    

'A Light in the Attic'

In [67]:
"""
product[0].find("p", attrs={"class":"price_color"}).get_text().replace("£","")
def get_price(item):
    brand_price=item.find("p", attrs={"class":"price_color"}).get_text().replace("£","")
    return brand_price
"""

'51.77'

In [71]:
product[0].find("p", attrs={"class":"instock availability"}).get_text().strip()
def get_avail(item):
    brand_avail=item.find("p", attrs={"class":"instock availability"}).get_text().strip()
    return brand_avail

'In stock'

In [78]:
product[0].find("p")["class"][1]

'Three'

In [ ]:
## The second step involves searching deep into the particular page for each item using the link filtered from the main or general page

In [ ]:
for prod in product():
    prod.find("img")["alt"]
    product[0].find("p", attrs={"class":"price_color"}).get_text().replace("£","")
    product[0].find("p", attrs={"class":"instock availability"}).get_text().strip()
    partial=product[0].find("a")["href"]
    domain="https://books.toscrape.com/"
    url=domain+partial